In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, Add, Activation, Flatten, Dense, add
from tensorflow.keras.models import Model

# MLflow 라이브러리 임포트
import mlflow
import mlflow.tensorflow

2025-09-12 03:00:37.632879: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-12 03:00:37.988626: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-12 03:00:37.988646: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-12 03:00:37.990601: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-12 03:00:38.151244: I tensorflow/core/platform/cpu_feature_g

원본 데이터

In [5]:
# 서버 내 학습데이터 디렉토리 경로
dataset_dir_path = Path("/home/jskwon/Desktop/project-nodal-surrogate/datasets/nodal-code-surrogate-data-registry/ASTRA/const_power_operation/first_cycle_depletion/iSMR/run_250814_cycle_length/processed_data")
artifacts_dir_path = Path("/home/jskwon/Desktop/project-nodal-surrogate/workspace/nodal-code-surrogate-pipeline/notebooks/temp-training-model-cycle_length/temp-artifacts")

# 특정 dir path + "파일명"
mac_xs_voxel_4d_filename = "mac_xs_voxel_4d_250814.npy"
cycle_length_csv_filename = "efpd_interpolation_results.csv"

mac_xs_voxel_4d = np.load(dataset_dir_path/mac_xs_voxel_4d_filename)
df_cycle_length = pd.read_csv(dataset_dir_path/cycle_length_csv_filename, index_col=0) 
cycle_length = df_cycle_length.to_numpy()

데이터 클린징
- cycle_length 데이터 중, 해당 값을 못찾은 케이스 존재

In [6]:
cycle_length

array([['1058.409090909091'],
       ['976.2857142857143'],
       ['1036.3636363636363'],
       ...,
       ['Summary Not Found'],
       ['1056.530612244898'],
       ['843.1034482758621']], dtype=object)

In [8]:
# 1. cycle_length 데이터를 처리하며 invalid_indice 기록

valid_data_list = []
invalid_indices_list = []

for idx, value in enumerate(cycle_length):
    # cycle_length 데이터가, 1차원 배열이여야 하지만 마지막 차원이 하나 더 존재하므로 제거
    item = value[0]
    
    # 문자열을 실수로 변환
        # 변환 시도 성공: 실수형 데이터
        # 변환 실패 : 문자열 문구가 기록된 리스트의 요소
    try:
        float_value = float(item)
        valid_data_list.append(float_value) # 변환 성공시 valid_data_list에 추가
    except ValueError:
        invalid_indices_list.append(idx) # 변환 실패시, invalid_indice_list에 기록

# 리스트들을 최종적인 NumPy 배열로 변환
clean_cycle_length = np.array(valid_data_list)
invalid_indices = np.array(invalid_indices_list)


# 2. invalid_indices를 이용해 mac_xs_4d를 제거

clean_mac_xs_voxel_4d = np.delete(mac_xs_voxel_4d,
                                  invalid_indices,
                                  axis = 0)

In [13]:
print("Shape of data after cleansing - mac_xs_voxel:" ,clean_mac_xs_voxel_4d.shape)
print("Shape of data after cleansing - cycle_length:", clean_cycle_length.shape)

Shape of data after cleansing - mac_xs_voxel: (9191, 24, 5, 5, 10)
Shape of data after cleansing - cycle_length: (9191,)


In [14]:
# 실험 이름 설정 (MLflow UI에서 보일 그룹 이름)
mlflow.set_experiment("mlflow_test_first_250912")

2025/09/12 04:40:23 INFO mlflow.tracking.fluent: Experiment with name 'mlflow_test_first_250912' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/jskwon/Desktop/project-nodal-surrogate/datasets/nodal-code-surrogate-data-registry/ASTRA/const_power_operation/first_cycle_depletion/iSMR/run_250814_cycle_length/processed_data/mlruns/335135521009550182', creation_time=1757652023016, experiment_id='335135521009550182', last_update_time=1757652023016, lifecycle_stage='active', name='mlflow_test_first_250912', tags={}>

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, Add, Activation, Flatten, Dense, add
from tensorflow.keras.models import Model

# MLflow 라이브러리 임포트
import mlflow
import mlflow.tensorflow

# ----------------------------------------
# 1. 데이터 로딩 (기존 코드와 동일)
# ----------------------------------------
# 서버 내 학습데이터 디렉토리 경로
dataset_dir_path = Path("/home/jskwon/Desktop/project-nodal-surrogate/datasets/nodal-code-surrogate-data-registry/ASTRA/const_power_operation/first_cycle_depletion/iSMR/run_250814_cycle_length/processed_data")
artifacts_dir_path = Path("/home/jskwon/Desktop/project-nodal-surrogate/workspace/nodal-code-surrogate-pipeline/notebooks/temp-training-model-cycle_length/temp-artifacts")

# 특정 dir path + "파일명"
mac_xs_voxel_4d_filename = "mac_xs_voxel_4d_250814.npy"
cycle_length_csv_filename = "efpd_interpolation_results.csv"

mac_xs_voxel_4d = np.load(dataset_dir_path / mac_xs_voxel_4d_filename)
df_cycle_length = pd.read_csv(dataset_dir_path / cycle_length_csv_filename, index_col=0)
cycle_length = df_cycle_length.to_numpy()

# (가정) 이 부분에 train/validation 데이터 분할 코드가 있다고 가정합니다.
# 예: from sklearn.model_selection import train_test_split
# x_train, x_valid, y_train, y_valid = train_test_split(mac_xs_voxel_4d, cycle_length, test_size=0.2)
# 이 예시에서는 변수들이 이미 있다고 가정하고 진행합니다.

# ----------------------------------------
# 2. 모델 정의 (기존 코드와 동일)
# ----------------------------------------
def residual_block_3d(input_tensor, num_filters, kernel_size):
    # First convolution
    x = Conv3D(num_filters, kernel_size=kernel_size, strides=(1, 1, 1), padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolution
    x = Conv3D(num_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = Activation('relu')(x)

    # Shortcut path
    shortcut = Conv3D(num_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(input_tensor)

    # Add shortcut to output of conv layers
    x = add([x, shortcut])
    x = Activation('relu')(x)
    return x

# ----------------------------------------
# 3. MLflow 실험 시작 및 모델 훈련
# ----------------------------------------
# 실험 이름 설정 (MLflow UI에서 보일 그룹 이름)
mlflow.set_experiment("cycle_length_prediction")

# MLflow 실행(Run) 시작
with mlflow.start_run(run_name="ResNet_lr1e-4_epochs150"):
    # --- MLflow를 위한 추가 로깅 ---
    # 데이터 소스 정보 로깅
    mlflow.log_param("dataset_dir", str(dataset_dir_path))
    mlflow.log_param("input_data_file", mac_xs_voxel_4d_filename)
    mlflow.log_param("target_data_file", cycle_length_csv_filename)
    
    # 하이퍼파라미터 정의 및 로깅
    learning_rate = 1e-4
    epochs = 150
    input_shape = (24, 5, 5, 10)

    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("input_shape", str(input_shape))
    
    # --- 모델 구성 및 컴파일 ---
    input_lp = Input(shape=input_shape, name='input_lp')
    x = residual_block_3d(input_lp, 99, (15, 5, 5))
    x = residual_block_3d(x, 99, (15, 3, 3))
    x = residual_block_3d(x, 99, (3, 3, 3))
    x = residual_block_3d(x, 1, (3, 3, 3))
    x_flatten = Flatten(name='flatten_features')(x)
    x_dense = Dense(128, activation='relu', name='dense_layer_1')(x_flatten)
    x_dense = Dense(64, activation='relu', name='dense_layer_2')(x_dense)
    output_scalar = Dense(1, name='output_scalar')(x_dense)

    model = Model(inputs=input_lp, outputs=output_scalar)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    
    # 모델 아키텍처를 텍스트 파일로 저장하여 아티팩트로 로깅
    summary_list = []
    model.summary(print_fn=lambda x: summary_list.append(x))
    model_summary = "\n".join(summary_list)
    mlflow.log_text(model_summary, "model_summary.txt")

    # TensorFlow Autologging 활성화
    mlflow.tensorflow.autolog()

    # --- 데이터 타입 변환 및 모델 학습 ---
    # (가정) x_train, y_train, x_valid, y_valid 변수가 준비되어 있어야 합니다.
    # x_train = np.array(x_train, dtype=np.float32)
    # y_train = np.array(y_train, dtype=np.float32)
    # x_valid = np.array(x_valid, dtype=np.float32)
    # y_valid = np.array(y_valid, dtype=np.float32)

    # history = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_valid, y_valid))

    # --- 기존 모델 저장 코드 (필요시 유지) ---
    # Autolog가 모델을 저장해주므로 이 코드는 사실상 불필요하지만, 백업용으로 둘 수 있습니다.
    # model.save("experiment-250911-cycle_length-v0.h5")

print("MLflow 로깅이 완료되었습니다.")

MLflow 로깅이 완료되었습니다.
